# Descarga de datos de hospitalizados desde Sanidad

## Objetivo

Vamos a hacer una lectura de los datos de hospitalizados publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) en página 3,  publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [1]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import fitz
# https://pypi.org/project/PyMuPDF/#description

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [2]:
fuente_ficheros_pdf_sanidad = '/Users/mharias/documents/proyectos/covid/sanidad_fallecidos/pdfs_sanidad/'
dir_csv = '/Users/mharias/documents/proyectos/covid/sanidad_hospital/data/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

## Descarga de los ficheros de Sanidad

### Función de para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [3]:
def descarga(url,num):
    print('Descargando:', url)
    fn=fuente_ficheros_pdf_sanidad+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [4]:
#for i in range(287,297):
#    descarga(URL_reg.format(i),i)

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [5]:
n_fichero = 339

In [6]:
descarga(URL_reg.format(n_fichero),n_fichero)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_339_COVID-19.pdf


True

## Parseado y preparación del pandas

In [7]:
columnas = ['fecha','ccaa','Hosp_Totales','Hosp_7días','UCI_Totales','UCI_7dias','Hosp_actual','%CamasCovid',
        'UCI_actual','%CamasUCI','Ingresos_24horas','Altas_24horas']
datos = pd.DataFrame(columns=columnas)

fichero auxiliar que mapea nº de informes con fechas....Mejora: automatizarlo

In [8]:
claves={}
indice=76
rango= np.arange(datetime(2020,4,16),datetime.today(),timedelta(days=1)).astype(datetime)
for day in rango:
    if not (((day>datetime(2020,6,30))&(day.weekday() in [5,6])) |\
            (day==datetime(2020,12,6)) | (day==datetime(2020,12,8)) | (day==datetime(2020,12,25)) | \
           (day==datetime(2021,1,1)) | (day==datetime(2021,1,6)) | (day==datetime(2021,3,19))):
        #print (day,i,)
        indice+=1
        claves[indice]=day.strftime('%-d/%-m/%Y')

In [9]:
comunidades=['Andalucía','Aragón','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
             'Castilla y León','Cataluña','Ceuta','C. Valenciana','Extremadura','Galicia',
             'Madrid','Melilla','Murcia','Navarra','País Vasco','La Rioja']

# antes 235

In [10]:
rango_informes = np.concatenate([np.arange(88,137),np.arange(138,234)])
#rango_informes = np.concatenate([np.arange(88,90)])
opt='text'
for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(1)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(1).replace('¥','').split('\n')[1:]
        # c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
                                                                       
        cadena = [i.rstrip().lstrip().replace('.','') if i not in ['',' '] else 0 for i in cadena]
        
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        c1,c2,c3,c4,c5,c6 = cadena
        datos = datos.append({'fecha':claves[informe],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        #print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

In [11]:
datos.head(20)

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
5,27/4/2020,Cantabria,995,4,78,0,NaN,NaN,NaN,NaN,NaN,NaN
6,27/4/2020,Castilla La Mancha,8444,27,579,14,NaN,NaN,NaN,NaN,NaN,NaN
7,27/4/2020,Castilla y León,7703,50,518,3,NaN,NaN,NaN,NaN,NaN,NaN
8,27/4/2020,Cataluña,25799,134,2724,141,NaN,NaN,NaN,NaN,NaN,NaN
9,27/4/2020,Ceuta,10,0,4,0,NaN,NaN,NaN,NaN,NaN,NaN


## desde 235

In [12]:
rango = np.concatenate([np.arange(235,266),np.arange(267,n_fichero+1)])
opt='text'
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{} '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ *.\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('¥','').replace('.','').replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('¥','').replace('.','').replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('¥','').replace('.','').replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('¥','').replace('.','').replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('¥','').replace('.','').replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('¥','').replace('.','').replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves[i],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

In [13]:
datos.tail(20)

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
4711,24/3/2021,La Rioja,3187,12,360,2,41,"5,67%",17,"32,08%",4,6
4712,25/3/2021,Andalucía,40525,330,4201,27,1049,"5,80%",247,"13,25%",121,134
4713,25/3/2021,Aragón,12763,109,1182,13,269,"6,41%",53,"22,46%",31,34
4714,25/3/2021,Asturias,7785,95,585,7,266,"7,65%",77,"23,62%",22,24
4715,25/3/2021,Baleares,4469,7,666,1,73,"2,35%",22,"8,12%",3,7
4716,25/3/2021,Canarias,4582,104,907,12,282,"5,16%",84,"17,61%",23,35
4717,25/3/2021,Cantabria,2726,39,374,8,58,"3,69%",16,"13,33%",11,10
4718,25/3/2021,Castilla La Mancha,18387,17,1343,2,186,"3,78%",47,"12,11%",38,21
4719,25/3/2021,Castilla y León,25406,118,2002,7,412,"6,10%",119,"21,72%",42,45
4720,25/3/2021,Cataluña,38006,181,3131,12,1837,"7,25%",452,"31,83%",205,208


In [14]:
datos.to_csv(dir_csv+'datos_sanidad_hosp.csv',index=False,index_label=False)

# Taller de reparación

In [15]:
rango = np.arange(326,327)
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[¥\*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ *.\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        print (comunidad)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('¥','').replace('.','').replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('¥','').replace('.','').replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('¥','').replace('.','').replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('¥','').replace('.','').replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('¥','').replace('.','').replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('¥','').replace('.','').replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves[i],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

Andalucía
Aragón
Asturias
Baleares
Canarias
Cantabria
Castilla La Mancha
Castilla y León
Cataluña
Ceuta
C. Valenciana
Extremadura
Galicia
Madrid
Melilla
Murcia
Navarra
País Vasco
La Rioja


In [16]:
datos

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4726,5/3/2021,Melilla,504,14,76,2,18,"9,89%",3,"17,65%",4,1
4727,5/3/2021,Murcia,8565,67,1101,11,172,"4,16%",66,"13,58%",14,19
4728,5/3/2021,Navarra,5152,27,451,5,70,"3,61%",14,"10,69%",4,4
4729,5/3/2021,País Vasco,12718,54,1074,8,494,"9,88%",110,"25,11%",65,72


In [17]:
text

' \n \n Centro de Coordinación de Alertas \ny Emergencias Sanitarias \n \nPágina 3 de 21 \n \nSECRETARÍA DE ESTADO \nDE SANIDAD \n \nDIRECCIÓN GENERAL DE \nSALUD PÚBLICA  \n \n \n \nTabla 2. Casos de COVID-19 que han precisado hospitalización e ingreso en UCI. \nTabla 3. Situación capacidad asistencial y actividad Covid-19 en  hospitales  \n \nCasos que han precisado \nhospitalización \nCasos que han ingresado en \nUCI \n \nCCAA \nTotal Pacientes \nCOVID \nhospitalizados \n% Camas \nOcupadas \nCOVID \nNúmero \nPacientes \nCOVID en \nUCI \n% Camas \nOcupadas \nUCI \nCOVID \nIngresos \nCOVID \núltimas \n24h \nAltas \nCOVID \núltimas \n24h \nCCAA \nTotal \nCon fecha de \ningreso en los \núltimos 7 días \nTotal \nCon fecha de \ningreso en UCI en \nlos últimos 7 días \n \nAndalucía \n38.413 \n333 \n3.857 \n20 \n \nAndalucía \n1.503 \n8,33% \n351 \n18,71% \n150 \n202 \nAragón \n12.391 \n154 \n1.126 \n11 \n \nAragón \n313 \n7,45% \n69 \n28,99% \n41 \n45 \nAsturias \n7.416 \n136 \n559 \n15 \n 